# Festival Playlists

In [1]:
import numpy as np
import pandas as pd
import requests
import json
import spotipy
from IPython.display import display

1. Use the Songkick API to get all the bands playing the festival
2. Use the Setlist.FM API to get the setlists
3. Use the Spotify API to create the playlists and add all the songs

### Set API credentials

In [2]:
with open('../auth.json') as f:
    credentials = json.load(f)

setlistfm_api_key = credentials['setlistfm']['api_key']
spotify_client_id = credentials['spotify']['client_id']
spotify_client_secret = credentials['spotify']['client_secret']

### Setlist FM

#### Plan of action
1. Given a lineup (list of band names), get their Musicbrainz identifiers (`mbid`) via `https://api.setlist.fm/rest/1.0/search/artists`
2. Retrieve the setlists for each artist using their `mbid` via `https://api.setlist.fm/rest/1.0/artist/{artist_mbid}/setlists
`

In [118]:
lineup = pd.read_csv(
    '../data/test_lineup.csv', header=None, names=['band'], encoding="ISO-8859-1"
)['band'].values

In [119]:
len(lineup)

188

In [120]:
lineup

array(['Gorillaz', 'The Strokes', 'Young Thug', 'Tame Impala',
       'Disclosure', 'Beck', 'Tyler, The Creator', 'The National',
       'Charli XCX', 'Bad Bunny', 'Iggy Pop', 'Jorja Smith', 'Metronomy',
       'Earl Sweatshirt', 'Caribou', 'Brockhampton', 'Jamie xx',
       'DJ Shadow', 'Kurt Vile', 'Yo La Tengo', 'FKA twigs',
       'Cigarettes After Sex', 'The Jesus and Mary Chain', 'Dinosaur Jr.',
       'King Krule', 'Chromatics', 'Pavement', 'Young Dolph',
       'Black Lips', 'Mabel', 'Freddie Gibbs', 'Khruangbin',
       'King Gizzard & The Lizard Wizard', 'DIIV', 'Rapsody', 'Madlib',
       'Bauhaus', 'Kano', 'Black Coffee', 'Desire', 'Little Simz',
       'Jamila Woods', 'IDLES', 'Autechre', 'Slowthai', 'C. Tangana',
       'Nina Kraviz', 'Pabllo Vittar', 'Napalm Death', 'Les Savy Fav',
       'Beach Bunny', 'Koffee', 'Rolling Blackouts Coastal Fever',
       'Big Freedia', 'Einst\x9frzende Neubauten', 'Shellac', 'Duki',
       'Mano Le Tough', 'Leon Vynehall', 'Shame', 'Fati

In [116]:
artists_url = 'https://api.setlist.fm/rest/1.0/search/artists'

In [121]:
lineup_mbids = []
not_found = []

for name in lineup:
    req = requests.get(artists_url,
                       headers={'x-api-key': setlistfm_api_key, 'Accept': 'application/json'},
                       params={'artistName': name, 'p': 1, 'sort': 'relevance'}
    )
    
    i = 0
    while (not req.ok) & (i <= 5):
        req = requests.get(artists_url,
                           headers={'x-api-key': setlistfm_api_key, 'Accept': 'application/json'},
                           params={'artistName': name, 'p': 1, 'sort': 'relevance'}
        )
        i += 1
    
    if req.ok:
        artist_response = req.json()['artist']
        num_artists = len(artist_response)
        if num_artists > 1:
            for i in range(num_artists):
                if artist_response[i]['name'].lower() == name.lower():
                    mbid = artist_response[i]['mbid']
                    lineup_mbids.append({'name': name, 'mbid': mbid})
                    break
        elif num_artists == 1:
            mbid = artist_response[0]['mbid']
            lineup_mbids.append({'name': name, 'mbid': mbid})
        elif num_artists == 0:
            print(f'No results I think for {name}')
        else:
            print(f'WTF {name}?')
                    
    else:
        print(f'Couldn\'t find {name}')
        not_found.append(name)

Couldn't find Bigklit
Couldn't find Sangre Nueva
Couldn't find Los Hijos De Yayo
Couldn't find Kampire & Decay
Couldn't find DJ Whoisdualipa
Couldn't find Tropical Fuckstorm
Couldn't find Chaqueta de chandal
Couldn't find Akasha Kid
Couldn't find Norsicaa
Couldn't find Kamma & Masalo
Couldn't find DJ Naranjito


In [122]:
lineup_mbids

[{'name': 'Gorillaz', 'mbid': 'e21857d5-3256-4547-afb3-4b6ded592596'},
 {'name': 'The Strokes', 'mbid': 'f181961b-20f7-459e-89de-920ef03c7ed0'},
 {'name': 'Young Thug', 'mbid': '800760de-bdf8-43a2-8fe0-44a2401a5515'},
 {'name': 'Tame Impala', 'mbid': '63aa26c3-d59b-4da4-84ac-716b54f1ef4d'},
 {'name': 'Disclosure', 'mbid': 'ae65c507-d9a0-4d42-9a6c-2b1f82158b9f'},
 {'name': 'Beck', 'mbid': '309c62ba-7a22-4277-9f67-4a162526d18a'},
 {'name': 'Tyler, The Creator',
  'mbid': 'f6beac20-5dfe-4d1f-ae02-0b0a740aafd6'},
 {'name': 'The National', 'mbid': '664c3e0e-42d8-48c1-b209-1efca19c0325'},
 {'name': 'Charli XCX', 'mbid': '260b6184-8828-48eb-945c-bc4cb6fc34ca'},
 {'name': 'Bad Bunny', 'mbid': '89aa5ecb-59ad-46f5-b3eb-2d424e941f19'},
 {'name': 'Iggy Pop', 'mbid': 'f37b3f31-b1f8-4b88-8cb5-b34f709b17d7'},
 {'name': 'Jorja Smith', 'mbid': 'd62614aa-ef2c-44cd-8a7b-91292755a1d0'},
 {'name': 'Metronomy', 'mbid': '93eb7110-0bc9-4d3f-816b-4b52ef982ec8'},
 {'name': 'Earl Sweatshirt', 'mbid': '8b22acd6-2

In [123]:
not_found

['Bigklit',
 'Sangre Nueva',
 'Los Hijos De Yayo',
 'Kampire & Decay',
 'DJ Whoisdualipa',
 'Tropical Fuckstorm',
 'Chaqueta de chandal',
 'Akasha Kid',
 'Norsicaa',
 'Kamma & Masalo',
 'DJ Naranjito']

In [125]:
artist_setlist = []

for a in lineup_mbids:
    songs_played = []
    mbid = a['mbid']
    setlists_url = f'https://api.setlist.fm/rest/1.0/artist/{mbid}/setlists'
    
    req = requests.get(setlists_url,
             headers={'x-api-key': setlistfm_api_key, 'Accept': 'application/json'},
             params={'p': 1}
    )
    
    i = 0
    while (not req.ok) & (i <= 5):
        req = requests.get(setlists_url,
                 headers={'x-api-key': setlistfm_api_key, 'Accept': 'application/json'},
                 params={'p': 1}
        )
        i += 1
            
    if req.ok:
    
        setlist_response = req.json()['setlist']
        num_setlists = len(setlist_response)

        for i in range(num_setlists):
            setlist = setlist_response[i]['sets']['set']
            num_sections = len(setlist)
            total_songs = []
            for p in range(num_sections):
                total_songs += setlist[p]['song']
            num_songs = len(total_songs)

            for i in range(num_songs):
                song = total_songs[i]
                song_title = song['name']
                # if the song is a cover add the original artist to the song title
                if 'cover' in song:
                    song_title += ' {}'.format(song['cover']['name'])
                songs_played.append(song_title)
                                           
        most_played_songs = list(pd.Series(songs_played).value_counts().head(15).index)

        artist_setlist.append({
            'artist': a['name'],
            'setlist': most_played_songs
        })
    else:
        not_found.append(a['name'])

In [126]:
not_found

['Bigklit',
 'Sangre Nueva',
 'Los Hijos De Yayo',
 'Kampire & Decay',
 'DJ Whoisdualipa',
 'Tropical Fuckstorm',
 'Chaqueta de chandal',
 'Akasha Kid',
 'Norsicaa',
 'Kamma & Masalo',
 'DJ Naranjito',
 'Mariah the Scientist',
 'La Favi',
 'DJ Fra',
 'DJ Kelvin',
 'Rombo',
 'Killavesi',
 'DJ Lizz',
 'Isabella',
 'DJ Playero',
 'Terrence Dixon',
 'Candlelight']

In [127]:
artist_setlist

[{'artist': 'Gorillaz',
  'setlist': ['On Melancholy Hill',
   'Humility',
   'Souk Eye',
   'Andromeda',
   'El Mañana',
   'Tomorrow Comes Today',
   'M1 A1',
   'Last Living Souls',
   'Every Planet We Reach Is Dead',
   'Tranz',
   'Lake Zurich',
   'Feel Good Inc.',
   'Dirty Harry',
   'Strobelite',
   'Clint Eastwood']},
 {'artist': 'The Strokes',
  'setlist': ['Someday',
   'Hard to Explain',
   'You Only Live Once',
   'Heart in a Cage',
   'Reptilia',
   'The Modern Age',
   'What Ever Happened?',
   'Last Nite',
   'New York City Cops',
   'Ize of the World',
   'Soma',
   'Juicebox',
   "I Can't Win",
   'Meet Me in the Bathroom',
   'Razorblade']},
 {'artist': 'Young Thug',
  'setlist': ['Best Friend',
   'With That',
   'Digits',
   'The London',
   'Killed Before Future & Young Thug',
   'Ecstasy',
   'Check',
   'With Them',
   'Pick Up the Phone Travis Scott & Young Thug',
   'On the Run',
   'Hot',
   'About the Money T.I.',
   'Lifestyle Rich Gang',
   'Relationship'

In [147]:
setlist_lengths = []
short_or_empty_setlist = []

for i in range(len(artist_setlist)):
    n_songs = len(artist_setlist[i]['setlist'])
    setlist_lengths.append({
        'artist': artist_setlist[i]['artist'],
        'n_songs': n_songs
    })
    
    if n_songs < 5:
        short_or_empty_setlist.append(artist_setlist[i]['artist'])

In [151]:
short_or_empty_setlist[:10]

['Black Coffee',
 'Autechre',
 'Nina Kraviz',
 'Mano Le Tough',
 'Fatima Yamaha',
 'Cuban Doll',
 'Amelie Lens',
 'Special Request',
 'Mica Levi',
 'Evian Christ']

### Spotify

In [149]:
username = 'adrialuz'
scope = 'playlist-modify-public'

token = spotipy.util.prompt_for_user_token(username, scope)

sp = spotipy.Spotify(auth=token)

sp.trace = False

In [158]:
sp.search(f'artist:{a}', limit=1, type='artist', market='GB')['artists']['items'][0]['id']

'3DrmwF7wWSSycjS1XWIIpb'

In [199]:
sp.search(
        f'artist:Khaled', limit=2, type='artist', market='GB'
    )['artists']['items']

2

In [204]:
spotify_ids = []
for a in short_or_empty_setlist:
    search_result = sp.search(
        f'artist:{a}', limit=5, type='artist', market='GB'
    )['artists']['items']
    
    if search_result:
        for i in range(len(search_result)):
            name = search_result[i]['name']
            if name.lower() == a.lower():
                artist_id = search_result[i]['id']
                spotify_ids.append(artist_id)
                break
            else:
                pass
    else:
        print(f'Couldn\'t find {a} on Spotify.')

Couldn't find Ben UFO on Spotify.
Couldn't find Jane Fitz on Spotify.


In [205]:
spotify_ids

['6wMr4zKPrrR0UVz08WtUWc',
 '6WH1V41LwGDGmlPUhSZLHO',
 '1oZmFNkGAT93yD1xX4vTRE',
 '04KmByEP6icXVY0PvJaMMp',
 '7eZRt08LoDy0nfIS6OwyMP',
 '32DuKytBMZn90iqyh2HEdc',
 '5Ho1vKl1Uz8bJlk4vbmvmf',
 '59xdAObFYuaKO2phzzz07H',
 '29LOCR81IrdEJjCAeCEOU3',
 '75BN2bMrpeLl89blBiaFuv',
 '5n30EjOcjTVG11vtfNKbsd',
 '3M3wTTCDwicRubwMyHyEDy',
 '4LSAINMy65w69vnT4xUq1b',
 '7mKwhB3UiepqzM946jBOyi',
 '6fFcUOFcbjeIuEomuUthkw',
 '41BaIYCli96MF2wqqOutBR',
 '4z8y2MjTFwLa73dABYP1io',
 '6tJNsGnrDvwoMmwv18wmUK',
 '6cSiT1s7Van5HiSu85m9MI',
 '6arFJeHXbAIl0Scu78M5Yz',
 '0XfQBWgzisaS9ltDV9bXAS',
 '1FVo44KTXqxo3JxXADWTd9',
 '3CkM2290WOa2ESzhlu5mzM',
 '28ztjHIXceRRntmTUfnmUX',
 '1cUt6hBRKlav4TIgjopnHC',
 '0zo109NM3S7CqHpvlXwqEN',
 '2LMaARRMktW5O2umBFaQUu',
 '0faVPXr9Sui0OWJl2hRDQT',
 '3UBeV2UkTunts46Pok3zQu',
 '2eIDAcLKnWc4D350YyzvgS',
 '3jisZjR3TBVb9tIfTbRHTS',
 '64kN9EkSTHYhda2FupL0KI',
 '7zpN81tVvPwlHcJSkSCyRa',
 '4I8qgsaz4mQa9ICeCPibIF',
 '6JACkt5SfzTM7JmtvBEmIk',
 '6gyhsZUy2fzzWZBRVPmOw4',
 '4ZG4S6skttdVhHz08AjMhR',
 

In [217]:
sp.artist('2kRYZkBRH3UsFlDd209DS5')['name']

'Zozo'

In [223]:
popular_songs = []

for artist_id in spotify_ids:
    search_results = sp.artist_top_tracks(artist_id, country='GB')['tracks']

    top_songs = []
    if search_results:
        for i in range(len(search_results)):
            song_name = search_results[i]['name']
            top_songs.append(song_name)
        popular_songs.append({
            'artist': sp.artist(artist_id)['name'],
            'setlist': top_songs
        })
    else:
        print(artist_id, sp.artist(artist_id)['name'])

2kRYZkBRH3UsFlDd209DS5 Zozo


In [220]:
popular_songs

[{'artist': 'Black Coffee',
  'songs': ['Drive (feat. Delilah Montagu) - Edit',
   'SBCNCSLY',
   'Embrace - Black Coffee + Aquatone Remix',
   'LaLaLa',
   'Your Eyes',
   'Turn Me On',
   'Wish You Were Here (feat. Msaki)',
   'Muyè - Black Coffee Remix',
   'Drive (feat. Delilah Montagu)',
   'Muyè (Black Coffee Remix)']},
 {'artist': 'Autechre',
  'songs': ['Bike',
   'Slip',
   'Clipper',
   'Yulquen',
   'Montreal',
   'VLetrmx',
   'Foil',
   'Dael',
   'Altibzz',
   'Nil']},
 {'artist': 'Nina Kraviz',
  'songs': ['Ghetto Kraviz - Original Mix',
   'Dream Machine',
   'Stranno Neobjatno',
   "I'm Gonna Get You - Original Mix",
   'Da',
   'Pain In The Ass - Original Mix',
   "Ghetto Kraviz - Regal 'Sad' Remix",
   'I Want You',
   'Pochuvstvui',
   'Feel My Butterfly - Jamie Jones Tribute To Aaron Carl Remix']},
 {'artist': 'Mano Le Tough',
  'songs': ['Primative People - Tale Of Us Remix',
   "Can't Do Without You - Tale of Us & Mano Le Tough Remix",
   'The Sea Inside',
   "Ev

Get the URI codes for each track

In [224]:
uris = []
missing_songs = []

for a in (artist_setlist + popular_songs):
    artist = a['artist']
    setlist = a['setlist']
    for s in setlist:
        s = s.replace(',', '').replace('\'', '').replace('"', '').replace('.', '').replace(
            '?', '').replace(')', '').replace('(', '').replace('/', '').replace(
            '\\', '').replace('&', '').replace('-', '')
        items = sp.search(q=f'artist:{artist} track:{s}', limit=1)['tracks']['items']
        if items:
            uri = items[0]['id']
            uris.append(uri)
        else:
            items = sp.search(q=f'track:{s}', limit=1)['tracks']['items']
            if items:
                if items != [None]:
                    uri = items[0]['id']
                    uris.append(uri)
            else:
                missing_songs.append({
                    'artist': artist,
                    'song': s
                })

In [225]:
len(uris)

1949

In [226]:
len(missing_songs)

51

In [227]:
missing_songs

[{'artist': 'Young Thug', 'song': 'Killed Before Future  Young Thug'},
 {'artist': 'Tame Impala', 'song': 'Mutant Gossip'},
 {'artist': 'Disclosure', 'song': 'Klambu Bassjackers  Apster'},
 {'artist': 'Charli XCX', 'song': 'Track 10  Blame It on Your Love'},
 {'artist': 'Jamie xx', 'song': 'Gosh  Nikes Frank Ocean'},
 {'artist': 'DJ Shadow', 'song': 'StemLong Stem'},
 {'artist': 'Pavement', 'song': 'PerfumeV'},
 {'artist': 'Young Dolph', 'song': 'A Hunnit Niggas and a Hunnit Racks'},
 {'artist': 'Black Lips', 'song': 'Look Here Satan'},
 {'artist': 'Khruangbin', 'song': 'Maria También  Misirlou Dick Dale'},
 {'artist': 'King Gizzard & The Lizard Wizard', 'song': 'SelfImmolate'},
 {'artist': 'King Gizzard & The Lizard Wizard', 'song': 'PeopleVultures'},
 {'artist': 'Rapsody',
  'song': '“If I Was Your Girlfriend” by Prince  A Roller Coaster Jam Called Love'},
 {'artist': 'Rapsody', 'song': 'Lailas Wisdom'},
 {'artist': 'Madlib', 'song': 'Auditorium Mos Def'},
 {'artist': 'Kano', 'song':

In [228]:
divisor = int(np.floor(len(uris) / np.ceil(len(uris) / 100)))
times = int(np.floor(len(uris) / divisor))

In [229]:
for i in range(times):
    subset = uris[divisor*i:divisor*(i+1)]
    sp.user_playlist_add_tracks(username, playlist_id='5sfuRJ3UgOv35RizTPycAC',
                                tracks=subset)